In [1]:
import os, json
import re, random
from tqdm import tqdm
from nltk import tokenize

In [3]:
ibm_pro_test = 'data/IBMDebate/filtered/test_ibm-nli-{}.json'.format('entailment')
ibm_con_test = 'data/IBMDebate/filtered/test_ibm-nli-{}.json'.format('contradiction')

with open(ibm_pro_test, 'r', encoding='utf-8') as f:
    entailment_test = json.load(f)

with open(ibm_pro_test, 'r', encoding='utf-8') as f:
    contradiction_test = json.load(f)

test = set()

for x in contradiction_test + entailment_test:
    test.add(x['premise'] + '***' + x['hypothesis'])

In [8]:
ibm_test_prompts = []
for x in test:
    formatted_x = {}
    formatted_x['premise'] = x.split('***')[0]
    formatted_x['hypothesis'] = x.split('***')[1]
    ibm_test_prompts.append(formatted_x)

In [11]:
with open('data/IBMDebate/filtered/ibm_test_prompts.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(ibm_test_prompts, indent=2))
    f.close()

In [ ]:
with open('flan-t5/chaosNLI_snli_augmented.json', 'r') as f:
    augmented_snli = json.load(f)

with open

In [2]:
args = {'label':'contradiction'}
path_train = 'data/IBMDebate/filtered/train_ibm-nli-{}.json'.format(args['label'])
path_dev = 'data/IBMDebate/filtered/dev_ibm-nli-{}.json'.format(args['label'])
path_test = 'data/IBMDebate/filtered/test_ibm-nli-{}.json'.format(args['label'])

with open(path_train, 'r', encoding='utf-8') as f:
    train = json.load(f)
    print(train)
with open(path_dev, 'r', encoding='utf-8') as f:
    dev = json.load(f)
    print(dev)
with open(path_test, 'r', encoding='utf-8') as f:
    test = json.load(f)
    print(test)

[{'premise': 'video game violence is not related to serious aggressive behavior in real life', 'explanation': 'a more recent 2009 study focusing specifically on serious aggressive behavior', 'hypothesis': 'This house believes that the sale of violent video games to minors should be banned'}, {'premise': 'video game violence is not related to serious aggressive behavior in real life', 'explanation': 'several major studies by groups such as The Harvard Medical School Center for Mental Health, The Journal of Adolescent Health, and The British Medical Journal have shown no conclusive link between video game usage and violent activity."', 'hypothesis': 'This house believes that the sale of violent video games to minors should be banned'}, {'premise': 'video game violence is not related to serious aggressive behavior in real life', 'explanation': 'John Sherry has said (2007) "Further, why do some researchers (e.g. Gentile & Anderson, 2003) continue to argue that video games are dangerous des

In [14]:
with open('data/IBMDebate/filtered/dev_ibm-nli-contradiction.json','r') as f:
    dev = json.load(f)
    print(dev)

[]


In [10]:
dev

[]

In [3]:
sensitives = ['This house believes atheism is the only way',
'This house would ban partial birth abortions',
'This house believes that Israel should lift the blockade of Gaza',
'This house supports the one-child policy of the republic of China',]
sensitives = [x.lower() for x in sensitives]
sensitives

['this house believes atheism is the only way',
 'this house would ban partial birth abortions',
 'this house believes that israel should lift the blockade of gaza',
 'this house supports the one-child policy of the republic of china']

In [4]:
def ascii_process(s):
    string_encode = s.encode('ascii','ignore')
    string_decode = string_encode.decode()
    return string_decode

In [9]:
t = "Ferguson adds that present research on violence in video games confirms no valid link, \u201cThe best studies now coming out \u2014 those which take care to use well-validated clinical outcome measures and which carefully control for other important risk factors \u2014 provide no support for the belief that VVG contributes to youth violence\u201d"
ascii_process(t)

'Ferguson adds that present research on violence in video games confirms no valid link, The best studies now coming out  those which take care to use well-validated clinical outcome measures and which carefully control for other important risk factors  provide no support for the belief that VVG contributes to youth violence'

In [5]:
ibm_nli_entailment = []
ibm_nli_contradiction = []
for d in debates:
    if not(d['premise'].lower() in sensitives):
        x = {}
        x['premise'] = ascii_process(d['claim'])
        x['explanation'] = ascii_process(d['reason'])
        x['hypothesis'] = ascii_process(d['premise'])
        if d['stance'] == 'PRO':
            ibm_nli_entailment.append(x)
        else:
            ibm_nli_contradiction.append(x)

In [17]:
with open('data/IBMDebate/filtered/ibm_nli_entailment.json','r') as f:
    ibm_nli_entailment = json.load(f)
    print(len(ibm_nli_entailment))
with open('data/IBMDebate/filtered/ibm_nli_contradiction.json','r') as f:
    ibm_nli_contradiction = json.load(f)
    print(len(ibm_nli_contradiction))

e_splits = [ibm_nli_entailment[:int(0.8*len(ibm_nli_entailment))], ibm_nli_entailment[int(0.8*len(ibm_nli_entailment)):int(0.9*len(ibm_nli_entailment))],ibm_nli_entailment[int(0.9*len(ibm_nli_entailment)):]]
c_splits = [ibm_nli_contradiction[:int(0.8*len(ibm_nli_contradiction))], ibm_nli_contradiction[int(0.8*len(ibm_nli_contradiction)):int(0.9*len(ibm_nli_contradiction))],ibm_nli_contradiction[int(0.9*len(ibm_nli_contradiction)):]]

splits = ['train', 'dev', 'test']


671
763


In [18]:
for i in range(len(splits)):
    with open('data/IBMDebate/filtered/{}_ibm-nli-entailment.json'.format(splits[i]),'w', encoding='utf-8') as f:
        f.write(json.dumps(e_splits[i], indent=2))
        f.close()

    with open('data/IBMDebate/filtered/{}_ibm-nli-contradiction.json'.format(splits[i]),'w', encoding='utf-8') as f:
        f.write(json.dumps(c_splits[i], indent=2))
        f.close()

In [46]:
with open('data/eSNLI/e-snli-neutral.json','r', encoding='utf-8') as f:
    data = json.load(f)

In [47]:
label ='entailment'
if label == 'entailment':
    prompt = 'Explain why the premise entails the hypothesis. '
elif label == 'contradiction':
    prompt = 'Explain why the premise contradicts the hypothesis. '
elif label == 'neutral':
    prompt = 'Explain why the premise is neutral with respect to the hypothesis. '

x = {}
filtered_data = []
for index in range(len(data)):
    try:
        x['input_text'] = prompt + 'Premise: ' + data[index]['premise'] + ' ' + 'Hypothesis: ' + data[index]['hypothesis']
        x['output_text'] = 'Explanation: ' + data[index]['explanation']
        x['premise'] = data[index]['premise']
        x['hypothesis'] = data[index]['hypothesis']
        filtered_data.append(data[index])
    except:
        print(data[index])
    


{'pairID': '7638876050.jpg#4r2n', 'premise': 'Cannot see picture to describe.', 'hypothesis': nan, 'explanation': nan, 'highlighted_premise': nan, 'highlighted_hypothesis': nan}
{'pairID': '112178718.jpg#2r5n', 'premise': 'A man with his surfboard stands in the sand.', 'hypothesis': 'fo', 'explanation': nan, 'highlighted_premise': nan, 'highlighted_hypothesis': nan}
{'pairID': '4878621384.jpg#4r1n', 'premise': 'Jumping with purple balls is so much fun!', 'hypothesis': nan, 'explanation': 'There is no comparison made with n/a.', 'highlighted_premise': 'Jumping with purple balls is so much fun!', 'highlighted_hypothesis': ' *n/a*'}
{'pairID': '651277216.jpg#1r1n', 'premise': 'A mom and her child walking down the street hand in hand in town.', 'hypothesis': 'an', 'explanation': nan, 'highlighted_premise': nan, 'highlighted_hypothesis': nan}
{'pairID': '4882073197.jpg#2r3n', 'premise': 'A woman with red shoes is standing in the middle of the street.', 'hypothesis': 'f', 'explanation': nan,

In [44]:
len(data)

189688

In [45]:
len(filtered_data)

189688

In [40]:
with open('data/eSNLI/e-snli-neutral.json','w', encoding='utf-8') as f:
    f.write(json.dumps(filtered_data, indent=2))
    f.close()

/home/xz479/explanation_gen


In [13]:
t = "Toddler is *helping* *to* *stir* *ingredients.*"

In [22]:
def find_highlighted(t):
    puncts = '!@#$%^&*()_+-=`~[]\{\};:",./<>?\''
    highlighted = re.findall(r'\*(.*?)\*',t)
    lowered = []
    for token in highlighted:
        for i in range(len(puncts)):
            token = token.replace(puncts[i], "")
        lowered.append(token.lower())
    return lowered

In [25]:
folder = 'data/chaosNLI/'
files = os.listdir(folder)
for file in files:
    with open(folder+file, 'r') as json_file:
        json_list = list(json_file)

    data = []
    for json_str in json_list:
        result = json.loads(json_str)
        data.append(result)
    
    with open(folder + file[:-1], 'w', encoding='utf-8') as f:
        f.write(json.dumps(data, indent=2))
        f.close()

In [3]:
labels = ['entailment', 'contradiction', 'neutral']
with open('data/chaosNLI/chaosNLI_snli.json','r') as f:
        original_data = json.load(f)


for l in labels:
    with open('flan-t5-prompted/results/results_{}.json'.format(l,l), 'r') as f:
        explanations = json.load(f)
    for i in range(len(explanations)):
        original_data[i]['example'][l+'_explanation'] = explanations[i]

with open('flan-t5-prompted/chaosNLI_snli_augmented.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(original_data, indent=2))
    f.close()

In [14]:
labels = ['entailment', 'contradiction']
with open('data/IBMDebate/filtered/ibm_test_prompts.json','r') as f:
        original_data = json.load(f)
for l in labels:
    with open('flan-t5/ibm_results/results/results_{}.json'.format(l), 'r') as f:
        explanations = json.load(f)
    
    for i in range(len(explanations)):
        original_data[i][l+'_explanation'] = explanations[i]


In [26]:
survey_data = []
labels = ['entailment', 'contradiction', 'neutral']
with open('flan-t5-prompted/chaosNLI_snli_augmented.json', 'r') as f:
    original_data = json.load(f)

original_data = original_data[:30]

for l in labels:
    for d in original_data:
        x = {}
        x['premise'] = d['example']['premise']
        x['hypothesis'] = d['example']['hypothesis']
        exp = d['example'][l+'_explanation']
        exp = tokenize.sent_tokenize(exp.strip('Explanation: '))[0]
        x['explanation'] = exp
        x['generation_label'] = l
        x['uid'] = d['example']['uid']
        survey_data.append(x)

with open('flan-t5-prompted/ibm_website.json', 'r') as f:
    ibm_data = json.load(f)

survey_data = survey_data + ibm_data
print(len(survey_data))

530


In [27]:
random.shuffle(survey_data)

In [28]:
with open('flan-t5/ibm_snli_small_website.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(survey_data, indent=2))
    f.close()

In [6]:
test = 'Explanation: People wait in line behind him is a rephrasing of a man selling donuts to a customer during a world exhibition event held in the city of Angeles. People are waiting in line.'

In [11]:
tokenize.sent_tokenize(test.strip('Explanation: '))[0]

'People wait in line behind him is a rephrasing of a man selling donuts to a customer during a world exhibition event held in the city of Angeles.'

In [9]:
test

'Explanation: People wait in line behind him is a rephrasing of a man selling donuts to a customer during a world exhibition event held in the city of Angeles. People are waiting in line.'

In [8]:
survey_data = []
labels = ['entailment', 'contradiction']
with open('data/IBMDebate/filtered/ibm_test_prompts.json', 'r') as f:
    original_data = json.load(f)
for l in labels:
    with open('flan-t5-prompted/ibm-results/results/results_{}.json'.format(l), 'r') as f:
        generated = json.load(f)
    for i in range(len(original_data)):
        exp = generated[i].split('Explanation: ')[-1]
        exp = tokenize.sent_tokenize(exp.strip('Explanation: '))[0]
        original_data[i]['{}_explanation'.format(l)] = exp
        original_data[i]['uid'] = 'ibm_' + str(i+1)

with open('flan-t5-prompted/ibm_augmented.json', 'w') as f:
    f.write(json.dumps(original_data, indent=2))
    f.close()